In [1]:
from numpy import *
from numpy.linalg import *
import matplotlib.pyplot as plt
from matplotlib import cm
from numba import jit

###### Flow definition #########################################################
#maxIter = 200000 # Total number of time iterations.
maxIter = 200 # Total number of time iterations.
Re      = 220.0  # Reynolds number.
nx = 520; ny = 180; ly=ny-1.0; q = 9 # Lattice dimensions and populations.
cx = nx/4; cy=ny/2; r=ny/9;          # Coordinates of the cylinder.
uLB     = 0.04                       # Velocity in lattice units.
nulb    = uLB*r/Re; omega = 1.0 / (3.*nulb+0.5); # Relaxation parameter.

###### Lattice Constants #######################################################
c = array([(x,y) for x in [0,-1,1] for y in [0,-1,1]]) # Lattice velocities.
t = 1.0/36.0 * ones(q)                                   # Lattice weights.

t[asarray([norm(ci)<1.1 for ci in c])] = 1.0 / 9.0
t[0] = 4.0 / 9.0

noslip = [c.tolist().index((-c[i]).tolist()) for i in range(q)] 
i1 = arange(q)[asarray([ci[0]<0  for ci in c])] # Unknown on right wall.
i2 = arange(q)[asarray([ci[0]==0 for ci in c])] # Vertical middle.
i3 = arange(q)[asarray([ci[0]>0  for ci in c])] # Unknown on left wall.

###### Function Definitions ####################################################
sumpop = lambda fin: sum(fin,axis=0) # Helper function for density computation.

@jit
def equilibrium(rho, u):              # Equilibrium distribution function.    
    # Slow
    #cu = 3.0 * dot(c, u.transpose(1,0,2))
    # No multi-thread
    #cu = 3.0 * einsum('ij,jkl', c, u)
    # Multi-thread
    cu = 3.0 * tensordot(c, u, axes=([1],[0]))
    
    usqr = 3.0 / 2.0 * (u[0]**2 + u[1]**2)    
    
    feq = zeros((q,nx,ny))
    for i in range(q): 
        feq[i,:,:] = rho * t[i] * (1. + cu[i] + 0.5 * (cu[i] ** 2) - usqr)

# Slower!
#    expr = (1. + cu + 0.5 * (cu ** 2) - usqr)    
#    feq = einsum('i,ijk->ijk', t, rho * expr)
    
    return feq

###### Setup: cylindrical obstacle and velocity inlet with perturbation ########
obstacle = fromfunction(lambda x,y: (x-cx)**2+(y-cy)**2<r**2, (nx,ny))
vel = fromfunction(lambda d,x,y: (1-d)*uLB*(1.0+1e-4*sin(y/ly*2*pi)),(2,nx,ny))
feq = equilibrium(ones((nx,ny)),vel); fin = feq.copy()
                
def solve(total_iter):
    ###### Main time loop ##########################################################
    for time in range(total_iter):
        
        fin[i1,-1,:] = fin[i1,-2,:] # Right wall: outflow condition.
        
        rho = sumpop(fin)           # Calculate macroscopic density and velocity.
        
        # Slow
        #u = dot(c.transpose(), fin.transpose((1,0,2)))/rho
        
        # Multi-thread
        u = tensordot(c, fin, axes=([0],[0])) / rho
        
        # No multi-thread
        #u = einsum('ij,ikl', c, fin) / rho   
        
        u[:,0,:] = vel[:,0,:] # Left wall: compute density from known populations.
        rho[0,:] = 1.0 / (1.0 - u[0,0,:]) * (sumpop(fin[i2,0,:]) + 2.0 * sumpop(fin[i1,0,:]))

        feq = equilibrium(rho,u) # Left wall: Zou/He boundary condition.
        fin[i3,0,:] = fin[i1,0,:] + feq[i3,0,:] - fin[i1,0,:]
        fout = fin - omega * (fin - feq)  # Collision step.

        for i in range(q): 
            fout[i, obstacle] = fin[noslip[i], obstacle]

        for i in range(q): # Streaming step.
            fin[i,:,:] = roll(roll(fout[i,:,:], c[i,0], axis=0), c[i,1], axis=1)
        
        if (time%100==0): # Visualization
            plt.clf(); plt.imshow(sqrt(u[0]**2+u[1]**2).transpose(),cmap=cm.Reds)
            plt.savefig("vel."+str(time/100).zfill(4)+".png")
            print "Saving time ", time   

In [2]:
solve(2)

Saving time  0
